In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras

Using TensorFlow backend.


In [2]:
df = pd.read_csv('creditcard.csv')
df.head(1)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0


In [3]:
df['Class'].unique() # 0 = no fraud, 1 = fraudulent

array([0, 1], dtype=int64)

In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [8]:
clf = Sequential([
    Dense(units=16, kernel_initializer='uniform', input_dim=30, activation='relu'),
    Dense(units=18, kernel_initializer='uniform', activation='relu'),
    Dropout(0.25),
    Dense(20, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

In [9]:
clf.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                496       
_________________________________________________________________
dense_2 (Dense)              (None, 18)                306       
_________________________________________________________________
dropout_1 (Dropout)          (None, 18)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                380       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                504       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 25        
Total params: 1,711
Trainable params: 1,711
Non-trainable params: 0
_________________________________________________________________


In [10]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, # for tensorboard
          write_graph=True, write_images=True)
clf.fit(X_train, Y_train, batch_size=15, epochs=5, callbacks=[tbCallBack])

Epoch 1/5
256326/256326 [==============================] - 89s - loss: 0.0073 - acc: 0.9990    
Epoch 2/5
256326/256326 [==============================] - 88s - loss: 0.0038 - acc: 0.9994    
Epoch 3/5
256326/256326 [==============================] - 88s - loss: 0.0035 - acc: 0.9994    
Epoch 4/5
256326/256326 [==============================] - 89s - loss: 0.0033 - acc: 0.9994    
Epoch 5/5
256326/256326 [==============================] - 89s - loss: 0.0032 - acc: 0.9994    


In [12]:
score = clf.evaluate(X_test, Y_test, batch_size=128)
print('\nAnd the Score is ', score[1] * 100, '%')

26880/28481 [===========================>..] - ETA: 0s
And the Score is  99.9297777466 %


In [13]:
model_json = clf.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
clf.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
